In [11]:
! pip install autogen-core

In [108]:
import venv
from pathlib import Path
import asyncio

from autogen import ConversableAgent
from autogen_core import CancellationToken
from autogen_core.code_executor import CodeBlock, CodeExecutor
from autogen_ext.code_executors.local import LocalCommandLineCodeExecutor


work_dir = Path("./coding")
work_dir.mkdir(exist_ok=True)

venv_dir = work_dir / ".venv"
venv_builder = venv.EnvBuilder(with_pip=True)
venv_builder.create(venv_dir)
venv_context = venv_builder.ensure_directories(venv_dir)

local_executor = LocalCommandLineCodeExecutor(work_dir=work_dir, virtual_env_context=venv_context)

print(
    await local_executor.execute_code_blocks(
        code_blocks=[
            CodeBlock(language="bash", code="pip install confluent-kafka")
        ],
        cancellation_token=CancellationToken(),
    )
)

CommandLineCodeResult(exit_code=0, output='', code_file='/home/jovyan/work/coding/tmp_code_ab6d451f70cd5423f935ed85230efbd66f55662a2a12c13d820cc75da78a005b.bash')


In [28]:
import re
from typing import List

from dataclasses import dataclass

from autogen_core import (
    MessageContext,
    RoutedAgent,
    SingleThreadedAgentRuntime,
    TopicId,
    TypeSubscription,
    message_handler,
    type_subscription,
)
from autogen_core.models import ChatCompletionClient, SystemMessage, UserMessage
from autogen_ext.models.openai import OpenAIChatCompletionClient

In [90]:
@dataclass
class Message:
    content: str

code_writer_topic = "CodeWriterAgent"
code_execution_topic = "CodeExecutionAgent"


In [91]:
@type_subscription(topic_type=code_writer_topic)
class CodeWriterAgent(RoutedAgent):
    def __init__(self, model_client: ChatCompletionClient) -> None:
        super().__init__("A code writer agent.")
        self._system_message = SystemMessage(
            content=(
                "You are a code generator generating python code"
            )
        )
        self._model_client = model_client

    @message_handler
    async def handle_user_description(self, message: Message, ctx: MessageContext) -> None:
        prompt = f"requirement description: {message.content}"
        llm_result = await self._model_client.create(
            messages=[self._system_message, UserMessage(content=prompt, source=self.id.key)],
            cancellation_token=ctx.cancellation_token,
        )
        response = llm_result.content
        assert isinstance(response, str)
        print(f"{'-'*80}\n{self.id.type}:\n{response}")
        
        await self.publish_message(Message(response), topic_id=TopicId(code_execution_topic, source=self.id.key))


In [92]:
def extract_markdown_code_blocks(markdown_text: str) -> List[CodeBlock]:
    pattern = re.compile(r"```(?:\s*([\w\+\-]+))?\n([\s\S]*?)```")
    matches = pattern.findall(markdown_text)
    code_blocks: List[CodeBlock] = []
    for match in matches:
        language = match[0].strip() if match[0] else ""
        code_content = match[1]
        code_blocks.append(CodeBlock(code=code_content, language=language))
    return code_blocks


@type_subscription(topic_type=code_execution_topic)
class CodeExecutionAgent(RoutedAgent):
    def __init__(self, code_executor: CodeExecutor) -> None:
        super().__init__("An executor agent.")
        self._code_executor = code_executor
        self._result = None

    @message_handler
    async def handle_message(self, message: Message, ctx: MessageContext) -> None:
        code_blocks = extract_markdown_code_blocks(message.content)
        if code_blocks:
            result = await self._code_executor.execute_code_blocks(
                code_blocks, cancellation_token=ctx.cancellation_token
            )
            print(f"\n{'-'*80}\nExecutor:\n{result.output}")
            self._result = result.output


In [93]:
model_client = OpenAIChatCompletionClient(
    model="gpt-4o-mini", 
    temperature=0.0
)

runtime = SingleThreadedAgentRuntime()

await CodeWriterAgent.register(
    runtime, type=code_writer_topic, factory=lambda: CodeWriterAgent(model_client=model_client)
)

await CodeExecutionAgent.register(
    runtime, type=code_execution_topic, factory=lambda: CodeExecutionAgent(code_executor=local_executor)
)

AgentType(type='CodeExecutionAgent')

In [94]:
runtime.start()

await runtime.publish_message(
    Message(content="tell me how many topics are in my broker with address redpanda:9092"),
    topic_id=TopicId(code_writer_topic, source="default"),
)

await runtime.stop_when_idle()

--------------------------------------------------------------------------------
CodeWriterAgent:
You can calculate the value of π (pi) using various methods. One common approach is to use the Monte Carlo method, which relies on random sampling. Below is a Python code snippet that demonstrates this method to estimate the value of π:

```python
import random

def calculate_pi(num_samples):
    inside_circle = 0

    for _ in range(num_samples):
        x = random.uniform(-1, 1)
        y = random.uniform(-1, 1)
        distance = x**2 + y**2
        
        if distance <= 1:
            inside_circle += 1

    # The area of the circle is π * r^2, and the area of the square is (2r)^2
    # Here, r = 1, so the area of the circle is π and the area of the square is 4.
    pi_estimate = (inside_circle / num_samples) * 4
    return pi_estimate

# Example usage
num_samples = 1000000
pi_value = calculate_pi(num_samples)
print(f"Estimated value of π using {num_samples} samples: {pi_value}")
```

In [105]:
from autogen_core import AgentId
agent_id = AgentId(code_execution_topic, "default")

agent = await runtime.try_get_underlying_agent_instance(agent_id)

In [106]:
agent._result

'Estimated value of π using 1000000 samples: 3.141652\n'